# ***Sentinel-2 L2A data access with the Planetary Computer STAC API***

## Importing tools for use in the data

In [ ]:
# suppress warnings
import warnings
warnings.filterwarnings('ignore')

# installing dependencies to the environment
!pip install rioxarray
!pip install pystac-client
!pip install stackstac
!pip install planetary-computer
!pip install odc-stac

# import GIS tools
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray as rio
import rasterio
from matplotlib.cm import RdYlGn,jet,RdBu

# import Planetary Computer tools
import stackstac
import pystac_client
import planetary_computer
from odc.stac import stac_load

## Discover and load data for analysis

In [ ]:
# define bounding box for entire data region (Latitude, Longitude)
lower_left = (40.75, -74.01)
upper_right = (40.88, -73.86)

In [ ]:
# calculate bounds for doing an archive data search
# bounds = (min_lon, min_lat, max_lon, max_lat)
bounds = (lower_left[1], lower_left[0], upper_right[1], upper_right[0])

In [ ]:
# define time wnidow
time_window = "2021-06-01/2021-09-01"

In [ ]:
# searching the Planetary Computer's STAC endpoint
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = stac.search(
    bbox=bounds,
    datetime=time_window,
    collections=["sentinel-2-l2a"],
    query={"eo:cloud_cover": {"lt": 30}},
)

In [ ]:
# finding number of scenes that touch our region
items = list(search.get_items())
print('This is the number of scenes that touch our region: ',len(items))

### load data into an xarray using stackstac

In [ ]:
# epsg = 4326, for longitude-latitude in degrees
signed_items = [planetary_computer.sign(item).to_dict() for item in items]

In [ ]:
# define pixel resolution for final product
# define scale according to selected crs, (i'll use degrees)
resolution = 10 # meters per pixel
scale = resolution / 111320.0 # degrees per pixel fpr crs=4326

In [ ]:
# load bands by odc-stac command
data = stac_load(
    items,
    bands=["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B11", "B12"],
    crs="EPSG:4326", # latitude-longitude
    resolution=scale, # Degrees
    chunks={"x": 2048, "y":2048},
    dtype="uint16",
    patch_url=planetary_computer.sign,
    bbox=bounds
)

In [ ]:
''' view the dimensions of our xarray and loaded variables
to ensure we have the right coordinates and spectral bands '''
display(data)

## Viewing real color images from the time series

In [ ]:
# plot sample images from the time series
plot_data = data[["B04", "B03", "B02"]].to_array()
plot_data.plot.imshow(col='time', col_wrap=4, robust=True, vmin=0, vmax=2500)
plt.show()

In [ ]:
# plot image for a single date
fig, ax = plt.subplots(figsize=(6,6))
plot_data.isel(time=7).plot.imshow(robust=True, ax=ax, vmin=0, vmax=2500)
ax.set_title("RGB Single Date: July 24, 2021")
ax.axis('off')
plt.show()

## Median composite

In [ ]:
median = data.median(dim="time").compute()

In [ ]:
# plot an image for median composite or mosaic
fig, ax = plt.subplots(figsize=(6,6))
#debuging
#print(type(median))
median[["B04", "B03", "B02"]].to_array().plot.imshow(robust=True, ax=ax, vmin=0, vmax=2500)
ax.set_title("RGB Median Composite")
ax.axis('off')
plt.show()
# the image is void of clouds due to statistical filtering

### Normalized Difference Vegetation Index (NDVI)

In [ ]:
# calculate NDVI for median mosaic
ndvi_median = (median.B08-median.B04)/(median.B08+median.B04)

In [ ]:
# plot image for ndvi median mosaic
fig, ax = plt.subplots(figsize=(6,6))
ndvi_median.plot.imshow(robust=True, ax=ax, cmap=RdYlGn, vmin=-1, vmax=1)
plt.title("NDVI median composite")
plt.axis('off')
plt.show()

### Normalized Difference Buildup Index (NDBI)

In [ ]:
# calculate NDBI for median mosaic
ndbi_median = (median.B11-median.B08)/(median.B11+median.B08)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
ndbi_median.plot.imshow(robust=True, ax=ax, cmap=RdYlGn, vmin=-0.1, vmax=0.1) #cmap="jet"
plt.title("NDBI median composite")
plt.axis('off')
plt.show()

### Normalized Difference Water Index

In [ ]:
# calculate NDWI for the median mosaic
ndwi_median = (median.B03-median.B08)/(median.B03+median.B08)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
ndwi_median.plot.imshow(robust=True, ax=ax, cmap=RdYlGn, vmin=-1, vmax=1)
plt.title("NDWI median composite")
plt.axis('off')
plt.show()

## Save output data in a GeoTIFF file

In [ ]:
filename = "S2_sample.tiff"

In [ ]:
# pick a single time slice July 24, 2021 (time=7)
data_slice = data.isel(time=7)

In [ ]:
# calculate dimensions of the file
# height = data_slice.dims["latitude"]
# width = data_slice.dims["longitude"]
height = data_slice.dims["latitude"]
width = data_slice.dims["longitude"]

In [ ]:
# Define CRS to be common Lat-Lon coordinates
# Define transformation using our bounding box, Lat-Lon info is written to the GeoTIFF
gt = rasterio.transform.from_bounds(lower_left[1], lower_left[0], upper_right[1], upper_right[0], width, height)
data_slice.rio.write_crs("epsg:4326", inplace=True)
data_slice.rio.write_transform(transform=gt, inplace=True)

In [ ]:
# create the GeoTIFF output file using the defined parameters
with rasterio.open(filename, 'w', driver='GTiff', width=width, height=height, crs='epsg:4326', transform=gt, count=11, compress='lzw', dtype='float64') as dst:
  dst.write(data_slice.B01,1)
  dst.write(data_slice.B02,2)
  dst.write(data_slice.B03,3)
  dst.write(data_slice.B04,4)
  dst.write(data_slice.B05,5)
  dst.write(data_slice.B06,6)
  dst.write(data_slice.B07,7)
  dst.write(data_slice.B08,8)
  dst.write(data_slice.B8A,9)
  dst.write(data_slice.B11,10)
  dst.write(data_slice.B12,11)
  dst.close()

In [ ]:
# show location ad size of new output file
!dir *.tiff